In [ ]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
import os
import cv2
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

#Note: No need to import Functional API separately in TensorFlow; Functional API built into core of tf.keras
#Create models using Functional API by directly using classes like Model, Input, and other layers from tensorflow.keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define base directory
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

# Verify each directory level
drive_dir = '/content/drive/MyDrive'
bootcamp_dir = os.path.join(drive_dir, 'BOOTCAMP')
colab_notebooks_dir = os.path.join(bootcamp_dir, 'ColabNotebooks')
project_dir = os.path.join(colab_notebooks_dir, 'ProjectWithGreg')
data_dir = os.path.join(project_dir, 'Data')

Mounted at /content/drive


In [ ]:
# Use os.path.join() to concatenate base_dir and 'train', set train_dir to 'Data/train'
# os.path.join(base_dir, 'train') takes base_dir, which = 'Data/', and concatenates it with string 'train'
train_dir = os.path.join(base_dir, 'train')

# Concatenate base_dir and 'test', set test_dir to 'Data/test'
# os.path.join(base_dir, 'test') takes base_dir, which = 'Data/', and concatenates it with string 'test'
test_dir = os.path.join(base_dir, 'test')

# Concatenate base_dir and 'valid', set valid_dir to 'Data/valid'
# os.path.join(base_dir, 'valid') takes base_dir, which = 'Data/', and concatenates it with string 'valid'
valid_dir = os.path.join(base_dir, 'valid')

#Read contents of base_dir directory and return list of names of entries (files and directories) in it
os.listdir(base_dir)

['valid',
 'test',
 'train',
 'best_chained_model_resnet.keras',
 'best_model_base_sparse.keras',
 'best_model_resnet_manual.keras',
 'best_resnet_manual_sparse.keras']

If you want to preprocess and feed your entire training set into the model (rather than use ImageDataGenerator

1. Structure Your Training Set: Assume training set is organized in directories, with each directory corresponding to a class label
2. Load and Preprocess the Training Set: write a function to load all images from the training set, preprocess them, and also load their corresponding labels  
3. Train the Model: use these preprocessed images and labels to train your model  

In this setup, training_set refers to all the images and their labels, loaded into arrays that can be used for training. The image_path would only refer to an individual image within this dataset if you were loading it separately for inference or testing purposes.  

Loading datasets manually versus using ImageDataGenerator can affect model accuracy, but the difference in accuracy will depend on:

1. Data Augmentation  
ImageDataGenerator is often used for data augmentation, which applies random transformations (e.g., rotations, flips, zooms) to the training images during training. This helps the model generalize better by seeing more varied data, leading to potentially higher accuracy, especially on unseen data.  

Without Augmentation: If you manually load and preprocess the data without any augmentation, the model will only see the original images, which might lead to overfitting and lower accuracy on validation or test sets.  

With Augmentation: If you use ImageDataGenerator with data augmentation, the model will see a broader range of data variations, improving generalization and potentially increasing accuracy.  

2. Real-Time Data Loading vs. Preloaded Data  
ImageDataGenerator loads and preprocesses images in real-time during training, which can handle large datasets more efficiently because it doesn't load the entire dataset into memory at once.  

Preloading All Data: If you manually load all images into memory before training, you might run into memory issues if your dataset is large. This method might also be less efficient in terms of memory usage and training speed.  

Real-Time Loading: ImageDataGenerator efficiently manages memory by loading batches of images during training. This can lead to smoother training processes, especially for large datasets, without impacting model performance due to memory constraints.  

3. Normalization and Preprocessing  
Both methods typically involve image normalization (e.g., scaling pixel values). If you ensure that the preprocessing steps are consistent (e.g., using preprocess_input for ResNet50), the difference in accuracy due to preprocessing alone should be negligible.  

4. Batch Size and Shuffling  
ImageDataGenerator handles batching and shuffling automatically, ensuring that the model sees different batches in a randomized order, which is important for training stability and accuracy.  

Manual Handling: If you manually load data, you need to ensure that batching and shuffling are implemented properly. Any inconsistency here could affect model performance.  

Summary:  
If you use data augmentation with ImageDataGenerator: You'll likely achieve better generalization and higher accuracy, especially on validation and test sets. If you don't use augmentation and manually load data: You might see lower accuracy due to overfitting, especially if your dataset is small or lacks variety.  
Preprocessing Consistency: As long as preprocessing is consistent, the difference in accuracy should not be significant if the other factors (like augmentation, shuffling, and memory management) are controlled for.  
If you prefer manual loading but still want the benefits of augmentation, you could manually apply augmentation using libraries like imgaug or albumentations before feeding the images into your model. However, using ImageDataGenerator is often more convenient and efficient for this purpose.  

You can manually load your datasets and add a data augmentation layer directly to the ResNet50 model (or any other model). This approach allows you to have the flexibility of manual data loading while still benefiting from data augmentation during training  


#Alternative Method: ImageDataGenerator (Not Used Here)
class in TensorFlow/Keras, powerful tool for real-time data augmentation and image preprocessing. Allows you to efficiently load, preprocess, and augment images in batches during training, which helps improve the generalization of your models.  

Key Features of ImageDataGenerator:
Data Augmentation: ImageDataGenerator can apply random transformations to your images to artificially expand your training dataset. This helps the model generalize better by seeing more varied data.

Examples of augmentations include:    
Rotation: Randomly rotates images by a specified degree range.
Width and Height Shifts: Randomly shifts the images horizontally or vertically.
Shear: Applies a shear transformation.  
Zoom: Zooms in or out on the images.  
Horizontal and Vertical Flip: Flips the images horizontally or vertically.  
Brightness Adjustment: Randomly adjusts the brightness of images.  
Channel Shifts: Randomly shifts the values in each channel.  

Real-Time Data Loading and Preprocessing: ImageDataGenerator loads and processes images in real-time during training, which means it doesn’t require you to load the entire dataset into memory at once. This is particularly useful for large datasets.  

Normalization: You can use ImageDataGenerator to normalize your images, for example, by rescaling pixel values (e.g., dividing by 255 to get values between 0 and 1).  

Batch Generation: It can generate batches of data (with corresponding labels) from directories or arrays, which is convenient for feeding data into model.

Flow Methods: ImageDataGenerator provides several methods to load and augment data:  
flow(x, y): Generates batches of data from numpy arrays.  
flow_from_directory(directory): Generates batches of data directly from a directory, where subdirectories are used as class labels.  
flow_from_dataframe(dataframe): Generates batches of data from a pandas DataFrame, with paths and labels defined in the DataFrame.  

In [ ]:
#SKIP THIS IF MANUALLY LOADING DATASETS, NOT USING IMAGEDATAGENERATOR

#Data generators are a convenient way to load and preprocess data in batches during model training
#ImageDataGenerator: This class from Keras's ImageDataGenerator module is used to generate batches of tensor image data with real-time data augmentation

# Define data generators for training and validation
# train_datagen and valid_datagen are instances of ImageDataGenerator used for training and validation data
# The rescale=1./255 parameter scales pixel values to the range [0,1]
#train_datagen = ImageDataGenerator(rescale=1./255)
#valid_datagen = ImageDataGenerator(rescale=1./255)

#img_size = (224, 224)  # Define target size for images
#batch_size = 32

# train_gen and valid_gen are actual data generators created using flow_from_directory method
# They load images from specified directories, rescale them, and convert labels to categorical format

# flow_from_directory method generates batches of augmented/normalized data from image files in a directory
# flow_from_directory method allows you to specify various parameters like target size, batch size, and class mode

#train_gen = train_datagen.flow_from_directory(
#    train_dir,
#    target_size=(img_size[0], img_size[1]),
#    batch_size=batch_size,
#    class_mode='categorical'
#)

#valid_gen = valid_datagen.flow_from_directory(
#    valid_dir,
#    target_size=(img_size[0], img_size[1]),
#    batch_size=batch_size,
#    class_mode='categorical'
#)

Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [ ]:
# import Image module from Python Imaging Library (PIL), library for opening, manipulating, and saving image file formats
# allows performing various operations like opening, resizing, cropping, enhancing, saving images, creating Image objects, loading images from files, manipulating images, performing image processing tasks

from PIL import Image
import matplotlib.pyplot as plt

# Define base directory
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'    #Directory Structure: Ensure base_dir points to directory where images stored within Google Drive

# Initialize dictionary to store images
images = {}

for root, _, files in os.walk(base_dir):          # Loop iterates over each tuple returned by os.walk(); root is current directory path; _ (underscore) is placeholder for directories within root; 'files' is list of files in current directory (root)
    for file in files:                            # Within each directory (root), iterate through each file
        if file.endswith('.png'):                 # Check current file ending to filter only PNG image files (can adjust to '.jpg', '.jpeg', any image file extension)
            file_path = os.path.join(root, file)  # Construct full path to image file by joining root directory path with current file name to give absolute file path of each image

            #Read image file using OpenCV (cv2), convert color space from BGR to RGB, and store in dictionary where file name serves as key

            img = cv2.imread(file_path)   # (1) Open image file using OpenCV cv2.
                                          # imread(file_path) reads image file specified by 'file_path' and loads it to NumPy array ('img')
                                          # cv2.imread(file_path) reads image from specified file path and returns it as NumPy array
                                          # OpenCV library is built on top of NumPy and uses NumPy arrays for image representation and manipulation
                                          # Array represents image in BGR (Blue-Green-Red) color format by default

            # (2) Convert BGR format to RGB format for displaying correctly with matplotlib

            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # (3) Store images ('img_rgb') in dictionary as values in key-value pairs, where 'files' = key
            # dictionary[file] = value    is syntax for accessing or assigning value ('img_rgb') to specific key ('file') within initialized dictionary ('images')
            # 'file' variable acts as key in dictionary; Each unique filename will be separate key in dictionary
            # img_rgb is value associated with key 'file'; a NumPy array representing image data in RGB format

            images[file] = img_rgb    # This line adds key-value pairs to 'images' {} dictionary, where 'file' is key and 'img_rgb' is value
                                      # Dictionaries in Python are collections of key-value pairs, where each key is unique

# Storing Data: executing images[file] = img_rgb tells Python to add or update an entry in images dictionary
# Unique Keys: Each unique file (filename) used as key ensures entries unique in dictionary (using same filename multiple times overwrites previous value associated with key)

# When images stored in dictionary with filename as key (images[file] = img_rgb), value associated with each key (the filename) is image data itself
# Key in dictionary (file) is string representing filename of image file being processed
# Value associated with each key is image data stored as NumPy array (img_rgb) containing pixel data of image after being read and processed (converted from BGR to RGB)

print(images.keys())

dict_keys(['000115 (5).png', '000115.png', '000116 (5).png', '000109 (3).png', '000116 (9).png', '000114.png', '000115 (2).png', '000112 (2).png', '000116 (3).png', '000113 (3).png', '000115 (9).png', '000117.png', '000112 (9).png', '000109 (8).png', '000108 (8).png', '000110 (7).png', '000109 (4).png', '000108 (7).png', '000113.png', '000114 (10).png', '000116 (8).png', '000117 (6).png', '000111 (2).png', '004162_01_01_150.png', '4 (2).png', '004007_01_01_519.png', '003828_02_01_174.png', '8 - Copy (3).png', '4 - Copy (2).png', '7.png', '6 - Copy.png', '6 - Copy (2) - Copy.png', '7 - Copy (3).png', '5.png', '6 - Copy (3).png', '7 - Copy (2).png', '000110.png', '000128.png', '000120.png', '000130.png', '000118 (2).png', '000112.png', '000108 (2).png', '000109.png', '000113 (2).png', '000110 (2).png', '000108.png', '000116.png', '000131.png', '000126.png', '000115 (3).png', '000122.png', '000111.png', '000119 (5).png', '000119.png', '000118 (5).png', '000116 (2).png', '000114 (4).png', 

In [ ]:
# Define base directory
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

# Verify each directory level
drive_dir = '/content/drive/MyDrive'
bootcamp_dir = os.path.join(drive_dir, 'BOOTCAMP')
colab_notebooks_dir = os.path.join(bootcamp_dir, 'ColabNotebooks')
project_dir = os.path.join(colab_notebooks_dir, 'ProjectWithGreg')
data_dir = os.path.join(project_dir, 'Data')

# Print contents at each level to ensure correctness
print("Contents of MyDrive:", os.listdir(drive_dir))
print("Contents of BOOTCAMP:", os.listdir(bootcamp_dir))
print("Contents of ColabNotebooks:", os.listdir(colab_notebooks_dir))
print("Contents of ProjectWithGreg:", os.listdir(project_dir))
print("Contents of Data:", os.listdir(data_dir))

Contents of MyDrive: ['New Recording 3.mp3', '2015 YEC Photos', 'CompetitionMatrix.xlsx.gsheet', 'Grad School Applications', 'Jussi', 'Photographs', 'Prospanica Materials', 'MicroEra Power Interest Form.gdoc', 'ALC Materials', 'Mitch Lyrics.gdoc', '2021 Holiday Card Versions', 'PaperlessPostAddresses.gsheet', 'Work Samples', 'Media.gdoc', 'CJ Paystub.gdoc', 'Energetic Insurance Longroad Portfolio Announcement_FINAL.docx', 'Chapter Finance Forms.xlsx', 'Marketing_Insurance_Digital.gdoc', 'Conexion 2022 Strengths Report (1).pdf', 'EC IV - MC XVII-Session 1 (Orientation)-Slides-02.02.2022 (1).pdf', 'EC IV - MC XVII-Session 1 (Orientation)-Slides-02.02.2022.pdf', 'Conexion 2022 Strengths Report.pdf', 'Copy of SLIDE Launch.gslides', 'Net Worth Statement.docx', 'Fact Checker.docx', 'MMG_Resume_03_22_2022.pdf', 'Resume 2022 LJL -2.pdf', 'Resume 2022 LJL -2 (1).gdoc', 'Resume 2022 LJL -2.gdoc', 'CoverLetter_Director of Communications_032322.pdf', 'CoverLetter_Director of Communications_032322.

Generate a tf.data.Dataset  
os module is powerful tool in Python for interacting with operating system
os module allows handling file and directory operations, managing environment variables, executing system commands, and more

os.path.join function joins one or more path components intelligently, adding appropriate directory separators (like '/' or '') depending on operating system to ensure resulting path is correctly formatted

After reading and processing all images, set up directory paths using os.path.join() based on base directory base_dir

base_dir = 'Data/' initializes base_dir as string containing path 'Data/'

Creating Directory Paths:
train_dir = os.path.join(base_dir, 'train') concatenates base_dir and 'train', resulting in train_dir being set to 'Data/train' test_dir = os.path.join(base_dir, 'test') concatenates base_dir and 'test', resulting in test_dir being set to 'Data/test' valid_dir = os.path.join(base_dir, 'valid') concatenates base_dir and 'valid', resulting in valid_dir being set to 'Data/valid'

Purpose:
Organizing data such as training, testing, and validation sets, within machine learning or data processing pipeline

Once directory paths defined, can use them with functions like os.listdir() to access files within each subset (train_dir, test_dir, valid_dir) for further processing, training models, or evaluating performance (file operations)

In [ ]:
#Set up directory paths using os.path.join() based on base directory base_dir
#For Organizing Data: directory paths typically used to organize data subsets, like training, testing, and validation sets, within machine learning or data processing pipeline
#For Directory Setup: base_dir, train_dir, test_dir, and valid_dir set up to organize different data subsets within 'Data/' directory
#For File Operations: once directory paths defined, use them with functions like os.listdir() to access files within subsets for further processing, training models, or evaluating performance

# Define base directory
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

# Use os.path.join() to concatenate base_dir and 'train', set train_dir to 'Data/train'
# os.path.join(base_dir, 'train') takes base_dir, which = 'Data/', and concatenates it with string 'train'

train_dir = os.path.join(base_dir, 'train')

# Concatenate base_dir and 'test', set test_dir to 'Data/test'
# os.path.join(base_dir, 'test') takes base_dir, which = 'Data/', and concatenates it with string 'test'

test_dir = os.path.join(base_dir, 'test')

# Concatenate base_dir and 'valid', set valid_dir to 'Data/valid'
# os.path.join(base_dir, 'valid') takes base_dir, which = 'Data/', and concatenates it with string 'valid'

valid_dir = os.path.join(base_dir, 'valid')

In [ ]:
#Read contents of base_dir directory and return list of names of entries (files and directories) in it

os.listdir(base_dir)

['valid',
 'test',
 'train',
 'best_chained_model_resnet.keras',
 'best_model_base_sparse.keras',
 'best_model_resnet_manual.keras',
 'best_resnet_manual_sparse.keras']

Create train, test and validation datasets  
What tf.keras.preprocessing.image_dataset_from_directory function does:  
1) Loads Images: function reads images from specified directory (train_dir)
Expects directory structure to follow certain format, where subdirectories represent different classes

2) Applies Transformations: images resized and batched into groups

3) Creates tf.data.Dataset: function returns tf.data.Dataset object, which is efficient dataset format provided by TensorFlow; object can be used directly in training and evaluation pipelines



'tf.keras.preprocessing.image_dataset_from_directory' is convenience function provided by TensorFlow’s Keras API to load image data from directory and prepare it for training or evaluation  

Automatically labels images based on directory structure and returns tf.data.dataset object that can be used in model training  
Function simplifies process of loading and preprocessing image data from directory  
Function labels images based on directory structure, resizes them, batches them, and returns tf.data.Dataset object for use in TensorFlow model training pipeline  

'train_dir' is directory where training images stored; should have specific structure, where subdirectories represent different classes  

'seed=101' for random operations like shuffling data; setting seed ensures reproducibility (ensures same shuffling order every time code runs)  

'image_size=(200, 200)' parameter specifies size to which each image will be resized (here, each image will resized to 200x200 pixels)  

'batch_size=32' defines number of images to be included in each batch; batching useful for efficient training, especially when datasets are large  

In [ ]:
# Using tf.keras.preprocessing.image_dataset_from_directory to generate training_set, testing_set, validation_set

# function is part of TensorFlow's Keras API
# creates tf.data.Dataset object from image files in directory; dataset object can be used for training model
# generates batches of augmented/normalized data from image files in directory; yields batches of data during training
# automatically handles tasks like loading images, resizing, and batching data

#Since you're using ResNet50, which expects input images of size (224, 224, 3), set image_size=(224, 224)

training_set = tf.keras.preprocessing.image_dataset_from_directory(     # image_dataset_from_directory method: images automatically labeled based on subdirectory names
                                                                        # each subdirectory treated as a class and labels assigned as integers starting from 0

train_dir,                  # Purpose: This is directory path where training images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )

# image_dataset_from_directory method: images automatically labeled based on subdirectory names
# each subdirectory treated as a class and labels assigned as integers starting from 0

testing_set = tf.keras.preprocessing.image_dataset_from_directory(
test_dir,                   # Purpose: This is directory path where test images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )

# image_dataset_from_directory method: images automatically labeled based on subdirectory names
# each subdirectory treated as a class and labels assigned as integers starting from 0

validation_set = tf.keras.preprocessing.image_dataset_from_directory(
valid_dir,                  # Purpose: This is directory path where valid images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )

Found 613 files belonging to 4 classes.
Found 315 files belonging to 4 classes.
Found 72 files belonging to 4 classes.


#Define Early Stopping  
Early Stopping in Machine Learning: technique used to prevent overfitting during ML model training  
Monitors model's performance on validation set and stops training process if model's performance stops improving  
This helps avoid overfitting, where model performs well on training data but poorly on unseen data  

In [ ]:
# Set up EarlyStopping callback in TensorFlow/Keras

from tensorflow.keras.callbacks import EarlyStopping    #Imports EarlyStopping class from TensorFlow's Keras API

callbacks = [EarlyStopping(patience=20)]                # Creates list of callbacks, with single EarlyStopping callback initialized with 'patience=20' parameter
                                                        # parameter specifies number of epochs with no improvement after which training will be stopped

# if patience=20, training process will continue for 20 more epochs after last improvement in monitored metric
# If no further improvement in last 20 epochs, training will be stopped early

#Define Data Augmentation  
'data_augmentation' function defines data augmentation pipeline using TensorFlow's Keras API  
Technique used to artificially increase size and diversity of training dataset by applying random transformations to input data  
Augmentaion helps improve generalization of model by making it more robust to variations in input data  

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom

#Note: OK to build some layers with Sequential API even if chained model built with Functional API

data_augmentation = Sequential([
    RandomFlip("horizontal"),#input_shape=(224, 224, 3)),    #defining input shape twice might cause conflicts depending on how model is serialized and reloaded
    RandomRotation(0.2),
    RandomZoom(0.2),
])

#DEFINE LAYERS OF MODEL AND BUILD MODEL

In [ ]:

#from tensorflow.keras import Model
#from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
#from tensorflow.keras.applications import ResNet50
#from tensorflow.keras.models import Sequential, Model
#from tensorflow.keras.optimizers import Adam
#import tensorflow as tf

# 1. Define input
#inputs = Input(shape=(224, 224, 3))

# 2. Load ResNet50 model without top layer
#base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=data_augmentation)    #Apply data augmentation

#When using include_top=False, ResNet50 outputs 4D tensor (feature maps) with shape (batch_size, height, width, channels)
#For ResNet50 with default configuration, shape will be (batch_size, 7, 7, 2048) because ResNet50’s last convolutional layer produces feature maps of size 7x7 with 2048 channels
# If output from ResNet50 base model is (batch_size, 7, 7, 2048), shape will be (batch_size, 7 * 7 * 2048) after flattening

# 3. Flatten output of ResNet50 model
#x = Flatten()(base_model.output)                            # Flatten layer converts 4D tensor into 2D tensor
                                                            # If output from ResNet50 base model is (batch_size, 7, 7, 2048), shape will be (batch_size, 7 * 7 * 2048) after flattening
                                                            # Flatten() flattens spatial dimensions and channels into single dimension
# 4. Add custom layers
#x = Dense(128, activation='relu')(x)                        # first Dense layer (Dense(128, activation='relu')) will process 1D feature vector of size 100352
#x = Dropout(0.25)(x)                                        # Dropout layer with 0.25 rate to prevent over fitting

#. Define output layer
#outputs = Dense(4, activation='softmax')(x)                 # number of units in Dense layer determines dimensionality of output space for layer
                                                            # doesn’t need to match number of features from previous layer but rather represents how many neurons (features) you want layer to have

# Build model
#manual_model = Model(inputs, outputs)

ValueError: Argument `input_tensor` must be a KerasTensor. Received invalid type: input_tensor=<Sequential name=sequential_4, built=False> (of type <class 'keras.src.models.sequential.Sequential'>)


ValueError: Argument `input_tensor` must be a KerasTensor. Received invalid type: input_tensor=<Sequential name=sequential_4, built=False> (of type <class 'keras.src.models.sequential.Sequential'>)

The error occurs because the input_tensor argument in ResNet50 expects a Keras tensor, which is typically created using the Input layer from the Functional API. However, you are passing a Sequential model (data_augmentation), which is not directly a Keras tensor.

To fix this, you need to first create an Input tensor and then pass it through the data augmentation layer. This way, the output of the data augmentation layer becomes a Keras tensor, which can then be used as the input_tensor for ResNet50.

In [ ]:
#CORRECTED CODE

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Create input tensor
inputs = Input(shape=(224, 224, 3))

# Data augmentation layer aleady defined using Sequential API; Apply data augmentation to input tensor
augmented_inputs = data_augmentation(inputs)

# Load ResNet50 model without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=augmented_inputs)

#When using include_top=False, ResNet50 outputs 4D tensor (feature maps) with shape (batch_size, height, width, channels)
#For ResNet50 with default configuration, shape will be (batch_size, 7, 7, 2048) because ResNet50’s last convolutional layer produces feature maps of size 7x7 with 2048 channels
# If output from ResNet50 base model is (batch_size, 7, 7, 2048), shape will be (batch_size, 7 * 7 * 2048) after flattening

# Add custom layers on top of ResNet50
x = Flatten()(base_model.output)                            # Flatten layer converts 4D tensor into 2D tensor, flattens spatial dimensions and channels into single dimension
x = Dense(128, activation='relu')(x)                        # first Dense layer (Dense(128, activation='relu')) will process 1D feature vector of size 100352
x = Dropout(0.25)(x)                                        # Dropout layer with 0.25 rate to prevent over fitting
# Define output layer
outputs = Dense(4, activation='softmax')(x)                 # number of units in Dense layer determines dimensionality of output space for layer
                                                            # doesn’t need to match number of features from previous layer but rather represents how many neurons (features) you want layer to have

#variable x represents output of each layer; x variable is updated at each step to reflect output of previous layer
#x variable reused at each stage to progressively update model as data passes through different layers

# Build model
manual_model = Model(inputs, outputs)                       #Model() class takes input tensor and output tensor to create model

# Compile model
manual_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print model summary
manual_model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential (Sequential)   │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_pad (ZeroPadding2D) │ (None, 230, 230, 3)    │              0 │ sequential[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,472 │ conv1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pad (ZeroPadding2D) │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1_pool (MaxPooling2D) │ (None, 56, 56, 64)     │              0 │ pool1_pad[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 64)     │          4,160 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 64)     │         36,928 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_bn         │ (None, 56, 56, 64)     │            256 │ conv2_block1_2_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_2_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_conv       │ (None, 56, 56, 256)    │         16,640 │ pool1_pool[0][0]       │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_3_conv  

 Total params: 36,433,412 (138.98 MB)

 Trainable params: 36,380,292 (138.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

The large number of layers in your model is due to the ResNet50 architecture. ResNet50 is a deep neural network with 50 layers, primarily consisting of convolutional, batch normalization, and activation layers. When you load it using include_top=False, all those layers are still included, minus the fully connected (dense) layers at the top.

Here's a breakdown of why you see so many layers:

ResNet50’s Architecture: ResNet50 is a deep model with residual blocks that contain multiple convolutional layers. Even though you load it without the top fully connected layers, the convolutional layers are still there.

Sequential Layers: Each residual block in ResNet50 contains several layers—convolutional, batch normalization, ReLU activations, and skip connections (additions)—which add to the total layer count.

Additional Layers Added by You: In your code, you add a flatten layer, a dense layer, a dropout layer, and the final dense output layer.

#Optimize, Callbacks, Prepare to Save, Compile, Model

In [ ]:
#Define optimizer
optimizer = Adam()

#Prepare to Save Model
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'
# Create base directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
# Define full file path including base directory
filepath = os.path.join(base_dir, 'best_resnet_manual_sparse.keras')

# Create ModelCheckpoint callbacks to save best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


#Compile, Train, Save, Reload Model

In [ ]:
# Compile model
manual_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train model with added callbacks
history = manual_model.fit(training_set, validation_data=validation_set, epochs=50, callbacks=checkpoint, verbose=1)

# Save model in specified directory
manual_model.save(filepath)

# Load model back
manual_model = load_model(filepath)

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.3636 - loss: 11.7031 
Epoch 1: val_accuracy improved from -inf to 0.18056, saving model to /content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data/best_resnet_manual_sparse.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 664s 30s/step - accuracy: 0.3656 - loss: 11.4459 - val_accuracy: 0.1806 - val_loss: 18213892.0000
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.3809 - loss: 1.2981 
Epoch 2: val_accuracy did not improve from 0.18056
20/20 ━━━━━━━━━━━━━━━━━━━━ 601s 29s/step - accuracy: 0.3824 - loss: 1.2939 - val_accuracy: 0.1806 - val_loss: 78964.0547
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 28s/step - accuracy: 0.4721 - loss: 1.3374 
Epoch 3: val_accuracy improved from 0.18056 to 0.26389, saving model to /content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data/best_resnet_manual_sparse.keras
20/20 ━━━━━━━━━━━━━━━━━━━━ 593s 30s/step - accuracy: 0.4731 - loss: 1.3319 - val_accuracy: 0.2639 - val_lo

Training stopped at 27/50 epochs. Reload model and restart training at 26th epoch.

In [ ]:
# REIMPORT necessary libraries
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import load_model

#REMOUNT Google Drive:
from google.colab import drive
drive.mount('/content/drive')

# REDEFINE base directory
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

# List contents of directory
!ls -l {base_dir}

# Verify each directory level
drive_dir = '/content/drive/MyDrive'
bootcamp_dir = os.path.join(drive_dir, 'BOOTCAMP')
colab_notebooks_dir = os.path.join(bootcamp_dir, 'ColabNotebooks')
project_dir = os.path.join(colab_notebooks_dir, 'ProjectWithGreg')
data_dir = os.path.join(project_dir, 'Data')

Mounted at /content/drive
total 1001718
-rw------- 1 root root  98159789 Aug 13 03:56 best_chained_model_resnet.keras
-rw------- 1 root root  52413377 Aug 13 19:14 best_model_base_sparse.keras
-rw------- 1 root root 437587362 Aug 14 21:13 best_model_resnet_manual.keras
-rw------- 1 root root 437585291 Aug 15 22:23 best_resnet_manual_sparse.keras
drwx------ 2 root root      4096 Jul 15 23:58 test
drwx------ 2 root root      4096 Jul 15 23:58 train
drwx------ 2 root root      4096 Jul 15 23:58 valid


In [ ]:
# REDEFINE EarlyStopping callback in TensorFlow/Keras

from tensorflow.keras.callbacks import EarlyStopping    #Imports EarlyStopping class from TensorFlow's Keras API

callbacks = [EarlyStopping(patience=20)]                # Creates list of callbacks, with single EarlyStopping callback initialized with 'patience=20' parameter
                                                        # parameter specifies number of epochs with no improvement after which training will be stopped

# if patience=20, training process will continue for 20 more epochs after last improvement in monitored metric
# If no further improvement in last 20 epochs, training will be stopped early

In [ ]:
#REDEFINE DATA AUGMENTATION

from tensorflow.keras import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, Flatten

data_augmentation = Sequential([
    RandomFlip("horizontal", input_shape=(224, 224, 3)),    #Set image dimensions to (224, 224, 3) for ResNet50
    RandomRotation(0.2),
    RandomZoom(0.2),
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
#REDEFINE MODEL

from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

# Create input tensor
inputs = Input(shape=(224, 224, 3))

# Data augmentation layer aleady defined using Sequential API; Apply data augmentation to input tensor
augmented_inputs = data_augmentation(inputs)

# Load ResNet50 model without top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=augmented_inputs)

#When using include_top=False, ResNet50 outputs 4D tensor (feature maps) with shape (batch_size, height, width, channels)
#For ResNet50 with default configuration, shape will be (batch_size, 7, 7, 2048) because ResNet50’s last convolutional layer produces feature maps of size 7x7 with 2048 channels
# If output from ResNet50 base model is (batch_size, 7, 7, 2048), shape will be (batch_size, 7 * 7 * 2048) after flattening

# Add custom layers on top of ResNet50
x = Flatten()(base_model.output)                            # Flatten layer converts 4D tensor into 2D tensor, flattens spatial dimensions and channels into single dimension
x = Dense(128, activation='relu')(x)                        # first Dense layer (Dense(128, activation='relu')) will process 1D feature vector of size 100352
x = Dropout(0.25)(x)                                        # Dropout layer with 0.25 rate to prevent over fitting
# Define output layer
outputs = Dense(4, activation='softmax')(x)                 # number of units in Dense layer determines dimensionality of output space for layer
                                                            # doesn’t need to match number of features from previous layer but rather represents how many neurons (features) you want layer to have

#variable x represents output of each layer; x variable is updated at each step to reflect output of previous layer
#x variable reused at each stage to progressively update model as data passes through different layers

# Build model
manual_model = Model(inputs, outputs)                       #Model() class takes input tensor and output tensor to create model

# Compile model
manual_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# RESPECIFY correct path to model

#base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

model_path = os.path.join(base_dir, 'best_resnet_manual_sparse.keras')

#Load saved model
manual_model = load_model(model_path)

In [ ]:
#Run this cell to redefine train_dir, test_dir, valid_dir

# Use os.path.join() to concatenate base_dir and 'train', set train_dir to 'Data/train'
# os.path.join(base_dir, 'train') takes base_dir, which = 'Data/', and concatenates it with string 'train'
train_dir = os.path.join(base_dir, 'train')

# Concatenate base_dir and 'test', set test_dir to 'Data/test'
# os.path.join(base_dir, 'test') takes base_dir, which = 'Data/', and concatenates it with string 'test'
test_dir = os.path.join(base_dir, 'test')

# Concatenate base_dir and 'valid', set valid_dir to 'Data/valid'
# os.path.join(base_dir, 'valid') takes base_dir, which = 'Data/', and concatenates it with string 'valid'
valid_dir = os.path.join(base_dir, 'valid')

#Read contents of base_dir directory and return list of names of entries (files and directories) in it
os.listdir(base_dir)

['valid',
 'test',
 'train',
 'best_chained_model_resnet.keras',
 'best_model_base_sparse.keras',
 'best_model_resnet_manual.keras',
 'best_resnet_manual_sparse.keras']

In [ ]:
#Run this cell to redefine training_set, testing_set, validation_set

# tf.keras.preprocessing.image_dataset_from_directory function generates tf.data.Dataset from image files in directory
# convenient way to load image data for training, validation, or testing in format that's easy to work with TensorFlow models

#FOR RESNET50 MODEL, RESIZE IMAGES to 224, 224, 3 (what ResNet based model expects)

training_set = tf.keras.preprocessing.image_dataset_from_directory(     # image_dataset_from_directory method: images automatically labeled based on subdirectory names
                                                                        # each subdirectory treated as a class and labels assigned as integers starting from 0

train_dir,                  # Purpose: This is directory path where training images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )


testing_set = tf.keras.preprocessing.image_dataset_from_directory(
test_dir,                   # Purpose: This is directory path where test images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )


validation_set = tf.keras.preprocessing.image_dataset_from_directory(
valid_dir,                  # Purpose: This is directory path where valid images are stored
                            # Structure: should contain subdirectories, each representing different class; name of each subdirectory will be used as class label for images within it
seed=101,
image_size=(224, 224),
batch_size=32,
label_mode='int'           # to work with sparse labels, use 'int' as value for label_mode parameter
    )

Found 613 files belonging to 4 classes.
Found 315 files belonging to 4 classes.
Found 72 files belonging to 4 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average

#Load saved model
manual_model = load_model(model_path)

train_score = manual_model.evaluate(training_set, verbose=1)
valid_score = manual_model.evaluate(validation_set, verbose=1)
test_score = manual_model.evaluate(testing_set, verbose=1)

# Print evaluation results
print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])

20/20 ━━━━━━━━━━━━━━━━━━━━ 124s 6s/step - accuracy: 0.4182 - loss: 1.9850
3/3 ━━━━━━━━━━━━━━━━━━━━ 17s 4s/step - accuracy: 0.4201 - loss: 1.8032
10/10 ━━━━━━━━━━━━━━━━━━━━ 63s 6s/step - accuracy: 0.2247 - loss: 5.0046
Train Loss:  1.9323302507400513
Train Accuracy:  0.44861337542533875
--------------------
Validation Loss:  1.8857617378234863
Validation Accuracy:  0.4027777910232544
--------------------
Test Loss:  5.019839286804199
Test Accuracy:  0.20952381193637848


The Flatten layer in a neural network model reshapes the input data into a 1D array, which is often necessary when transitioning from convolutional layers to fully connected layers in a deep learning model. The purpose of the Flatten layer is to flatten the input data so that it can be fed into the subsequent dense layers without changing the data itself.

When reloading a model and continuing its training, the model architecture and layer configurations need to match exactly to the original model. If the model includes a Flatten layer, it will expect the input data to be flattened when resuming training. If the input data provided during the reload and continuation of training does not match the expected flattened shape, it can lead to errors or unexpected behavior during training.

Here are some common issues that can arise when reloading a model with a Flatten layer for training continuation:

Input Shape Mismatch: If the input data provided during training continuation does not match the expected flattened shape after the Flatten layer, it can lead to shape mismatch errors.

Data Transformation: The Flatten layer transforms the input data, and if this transformation is not accounted for when reloading the model, the data fed into the model during training continuation may not be in the correct format.

Layer Compatibility: The presence of the Flatten layer can affect the compatibility of the model architecture with the data provided for training continuation. Any changes to the input data shape or preprocessing steps can impact the model's ability to resume training seamlessly.

To address issues related to the Flatten layer when reloading and continuing training, ensure that the input data provided matches the expected flattened shape after the Flatten layer and that any necessary preprocessing steps are applied consistently. Additionally, verifying the model architecture and input data compatibility before resuming training can help prevent errors related to the Flatten layer.